# GoodReads scraping awards data

In [ ]:
from bs4 import BeautifulSoup
import requests
import time, os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains

chromedriver = "C:/Users/Hp/Desktop/chromedriver.exe" 
os.environ["webdriver.chrome.driver"] = chromedriver

In [ ]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"profile.managed_default_content_settings.images": 2}
chromeOptions.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(chromedriver, chrome_options=chromeOptions)
driver.get("https://www.goodreads.com/choiceawards/best-fiction-books-2019")

# Function bookInfo to get each book's information

In [ ]:
def bookInfo():
    # Title
    Book_title= driver.find_element_by_xpath('//*[@id="bookTitle"]').text
    title.append(Book_title)
    
    # Author
    Author_name = driver.find_element_by_xpath('//*[@id="bookAuthors"]/span[2]/div/a/span').text #Author_name
    author.append(Author_name)
    
    
    # Number of Pages
    no_pages= driver.find_element_by_xpath('//span[@itemprop="numberOfPages"]')  #pages
    pages = np.int((no_pages.text).replace(' ','').replace('pages',''))    
    page.append(pages)
    
    # Year
    
    year = driver.find_element_by_xpath('//*[@id="details"]/div[2]').text 
    year_.append(year.split(' ')[4-1])

    
    # Click More Details
    driver.find_element_by_xpath('//*[@id="bookDataBoxShow"]').click()
    

        # ISBN
    ISBN = driver.find_element_by_xpath('//*[@id="bookDataBox"]/div[2]/div[2]').text
    ISBN_.append(ISBN[:10])
    
    
    # Number of Ratings
    no_ratings = driver.find_element_by_xpath('//*[@id="bookMeta"]/a[2]').text #ratings
    ratings = np.int(no_ratings.replace('ratings','').replace(' ','').replace(',',''))
    rating.append(ratings)
    
    
    # Rating Average
    avg_rating = np.float(driver.find_element_by_xpath('//*[@id="bookMeta"]/span[2]').text) #avg_rating
    average_r.append(avg_rating)
    
    
    # Click Rating Details
    button = driver.find_element_by_xpath('//*[@id="rating_details"]')
    driver.implicitly_wait(10)
    ActionChains(driver).move_to_element(button).click(button).perform()
    
        
        # Added by
    added= np.int(driver.find_element_by_xpath('//*[@id="moreBookData"]/table[2]/tbody/tr[1]/td[2]/span[4]').text)
    added_by.append(added)
    
    
        # Liked by %
    liked= (np.float(driver.find_element_by_xpath('//*[@id="moreBookData"]/span').text))/100 #liked
    liked_by.append(liked)
    
        # Number of reviews (comments)
    review= np.int(driver.find_element_by_xpath('//*[@id="moreBookData"]/table[2]/tbody/tr[1]/td[2]/span[3]').text)
    review_.append(review)

# To get the links for each Genre

In [ ]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
link_for_each_genre= []
p_tags = soup.find_all('a',class_="categoriesList__categoryLink")
for i,p_tags in enumerate(p_tags):
    link_for_each_genre.append(p_tags.get('href'))    
link_for_each_genre

# Loop for getting the books information from a whole year (All Genres)

In [ ]:
votes = []
title = [] 
author = [] 
page = []
ISBN_ = []
rating = []
average_r = []
added_by = []
liked_by = []
year_ = []
list_=[]
review_=[]


for i in range(0,len(link_for_each_genre)):
    
    element =  driver.find_element_by_xpath(f"//a[@href='{link_for_each_genre[i]}']")
    driver.execute_script("arguments[0].click();", element)
    
    book_tags=[]
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    book_tags = soup.find_all('a',class_="pollAnswer__bookLink")
    link_for_each_book= []
    
    for j,book_tags in enumerate(book_tags):
        link_for_each_book.append(book_tags.get('href'))    
    
    p_tags=[]   
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    p_tags = soup.find_all('strong',class_="uitext")
        
    for e,p_tags in enumerate(p_tags): 
        votes.append(p_tags.text)
    
    
    for k in range(0,len(link_for_each_book)):
        
        element =  driver.find_element_by_xpath(f"//a[@href='{link_for_each_book[k]}']")
        driver.execute_script("arguments[0].click();", element)
        bookInfo()
        driver.back()
        driver.implicitly_wait(10)
        driver.execute_script("window.scrollTo(0, window.scrollY + 60)")
    

        #votes = [np.int(i) for i in votes] 

## Cleaning votes 

In [ ]:
votes_=[]
for e in votes:
    votes_.append(e.replace('\n','').replace('votes','').replace(',',''))

# Saving

## Saving the information in a Dictionary

In [ ]:
GR_dict_ ={}
GR_dict_["Title"] = title
GR_dict_["Author"]= author
GR_dict_["ISBN"] = ISBN_
GR_dict_["Year"]= year_
GR_dict_["Pages"] = page
GR_dict_["No of Rating"] = rating
GR_dict_["Avg Rating"]= average_r
GR_dict_["Added by "] = added_by
GR_dict_["Liked by "] = liked_by
GR_dict_["Reviews"]= review_
GR_dict_["Votes"]= votes_

## Saving information in a dataframe

In [ ]:
df_ = pd.DataFrame(GR_dict_)
df_

## Saving the dataframe as a CSV

In [ ]:
df_.to_csv("GoodReads_Awards_dataset",index=False)